In [1]:
import os

In [21]:
X = []
Y = []
width = 100
height = 100 # resize to 100x100 to solve the RAM issue in Kaggle notebooks
INPUT_CLASSES_DIR = 'face_age/'

import numpy as np

for folder_name,_,filenames in os.walk(INPUT_CLASSES_DIR):
    #if folder_name !="face_age" and folder_name != 'Data':
    if folder_name !="face_age":
        for file in filenames:
            file_path = folder_name +"/"+ file
            image = Image.open(file_path).image.convert('RGB').resize((width, height))
            X.append(np.array(image))
            Y.append(int(folder_name[-3:]))
    else:
        pass

In [24]:
import pandas as pd

lst = list(zip(X, Y))

df = pd.DataFrame(lst, columns = ['image', 'age'])

df.head()

,image,age
0,"[[[147, 100, 82], [168, 118, 100], [179, 122, ...",1
1,"[[[239, 238, 233], [238, 235, 230], [241, 236,...",1
2,"[[[211, 211, 213], [220, 220, 222], [205, 205,...",1
3,"[[[193, 134, 94], [192, 134, 93], [189, 130, 9...",1
4,"[[[49, 38, 22], [40, 26, 17], [37, 18, 14], [3...",1


In [27]:
df.to_csv('dataframe.csv')